In [1]:
import re

In [2]:
!apt-get update -qq > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.3.3/spark-3.3.3-bin-hadoop3.tgz
!tar xzf spark-3.3.3-bin-hadoop3.tgz
!pip install -q findspark

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.3-bin-hadoop3"

import findspark
findspark.init()

from pyspark import SparkContext, SparkConf

spark_conf = SparkConf()\
  .setAppName("YourTest")\
  .setMaster("local[*]")

sc = SparkContext.getOrCreate(spark_conf)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

In [9]:
import nltk
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

def get_sentiment(stock):
  fitered_news = sc.textFile('/content/drive/MyDrive/trimmed_1_to_4.csv') \
                  .map(lambda x: re.sub(r"[^a-zA-Z0-9]\s", " ", x).lower()) \
                  .filter(lambda x: stock in x) \
                  .map(lambda x: x.split('|')) \
                  .filter(lambda x: len(x) >= 2) \
                  .map(lambda x: (x[0], ' '.join(x[1:]))) \
                  .map(lambda x: (x[0], x[1].split(' '))) \
                  .map(lambda x: (x[0], [re.sub(r'[^\w\s]', '', s).strip() for s in x[1]])) \
                  .map(lambda x: (x[0], [s for s in x[1] if s not in stop_words and s != '']))
  print("# of news on {}: {}".format(stock, len(fitered_news.collect())))

  def score(x):
      analyzer = SentimentIntensityAnalyzer()
      words = []
      for word in x:
          words.append(word)
      sentence = " ".join(words)
      # return TextBlob(sentence).sentiment.polarity
      return analyzer.polarity_scores(sentence)['compound']

  scores = fitered_news.map(lambda x: (x[0], score(x[1]))) \
                      .filter(lambda x: x[1] != 0) \
                      .reduceByKey(lambda x, y: x + y)

  return dict(scores.collect())

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [11]:
import json

sentiment = {
    'Facebook': get_sentiment('facebook'),
    'Lyft': get_sentiment('lyft'),
    'Netflix': get_sentiment('netflix'),

}

with open("/content/drive/MyDrive/sentiment.json", "w") as outfile:
    json.dump(sentiment, outfile)

# of news on facebook: 13428
# of news on lyft: 815
# of news on netflix: 3538
